<h1><center> ETL Pipeline application for supervised regression problem</center></h1>

# Lesson Goals
<div text-align:justify>our goal is  to accurately predict power output given a set of environmental readings from various sensors in a natural gas-fired power generation plant</div>

 

# Prerequests 
  1. Signup for databricks community cloud
  2. import the data from S3 or locally
  
  NB:make sure your cluster  running

In [0]:
labV1 = 'power-plant'

d Extract-Transform-Load (ETL) Your Data

In [0]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/10900_8-35d16.txt,10900_8-35d16.txt,4470586
dbfs:/FileStore/tables/10900_8.txt,10900_8.txt,4470586
dbfs:/FileStore/tables/2800.txt,2800.txt,1080847
dbfs:/FileStore/tables/Folds5x2_pp-1.xlsx,Folds5x2_pp-1.xlsx,2029440
dbfs:/FileStore/tables/Folds5x2_pp-2.xlsx,Folds5x2_pp-2.xlsx,2029440
dbfs:/FileStore/tables/Folds5x2_pp.ods,Folds5x2_pp.ods,1723202
dbfs:/FileStore/tables/Folds5x2_pp.xlsx,Folds5x2_pp.xlsx,2029440
dbfs:/FileStore/tables/PS_20174392719_1491204439457_log.csv,PS_20174392719_1491204439457_log.csv,72493779
dbfs:/FileStore/tables/advertising.csv,advertising.csv,4061
dbfs:/FileStore/tables/powerdata-1.csv,powerdata-1.csv,1591255


In [0]:
powerdf = spark.read.csv("/FileStore/tables/powerdata.csv", header="true")

In [0]:
powerdf.take(10)

Out[7]: [Row(AT='14.96', V='41.76', AP='1024.07', RH='73.17', PE='463.26'),
 Row(AT='25.18', V='62.96', AP='1020.04', RH='59.08', PE='444.37'),
 Row(AT='5.11', V='39.4', AP='1012.16', RH='92.14', PE='488.56'),
 Row(AT='20.86', V='57.32', AP='1010.24', RH='76.64', PE='446.48'),
 Row(AT='10.82', V='37.5', AP='1009.23', RH='96.62', PE='473.9'),
 Row(AT='26.27', V='59.44', AP='1012.23', RH='58.77', PE='443.67'),
 Row(AT='15.89', V='43.96', AP='1014.02', RH='75.24', PE='467.35'),
 Row(AT='9.48', V='44.71', AP='1019.12', RH='66.43', PE='478.42'),
 Row(AT='14.64', V='45', AP='1021.78', RH='41.25', PE='475.98'),
 Row(AT='11.74', V='43.56', AP='1015.14', RH='70.72', PE='477.5')]

In [0]:
powerdf.show(10)

+-----+-----+-------+-----+------+
 AT| V| AP| RH| PE|
+-----+-----+-------+-----+------+
14.96|41.76|1024.07|73.17|463.26|
25.18|62.96|1020.04|59.08|444.37|
 5.11| 39.4|1012.16|92.14|488.56|
20.86|57.32|1010.24|76.64|446.48|
10.82| 37.5|1009.23|96.62| 473.9|
26.27|59.44|1012.23|58.77|443.67|
15.89|43.96|1014.02|75.24|467.35|
 9.48|44.71|1019.12|66.43|478.42|
14.64| 45|1021.78|41.25|475.98|
11.74|43.56|1015.14|70.72| 477.5|
+-----+-----+-------+-----+------+
only showing top 10 rows

In [0]:
display(powerdf)

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


In [0]:
rawTextRdd = sc.textFile("/FileStore/tables/powerdata.csv")

In [0]:
mappedRdd= rawTextRdd.map(lambda x:x.split(";"))

In [0]:
mappedRdd.take(5)

Out[12]: [['AT,V,AP,RH,PE'],
 ['14.96,41.76,1024.07,73.17,463.26'],
 ['25.18,62.96,1020.04,59.08,444.37'],
 ['5.11,39.4,1012.16,92.14,488.56'],
 ['20.86,57.32,1010.24,76.64,446.48']]

In [0]:
sqlContext.sql("DROP TABLE IF EXISTS powerdf")

Out[13]: DataFrame[]

In [0]:
sqlContext.registerDataFrameAsTable(powerdf, "pplant")

In [0]:
%sql
-- We can use %sql to query the rows
SELECT * FROM pplant

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


In [0]:
%sql
desc pplant

col_name,data_type,comment
AT,string,null
V,string,null
AP,string,null
RH,string,null
PE,string,null


In [0]:
df = sqlContext.table("pplant")
display(df.describe())

summary,AT,V,AP,RH,PE
count,47840,47840,47840,47840,47840
mean,19.651231187290833,54.30580372073524,1013.2590781772577,73.3089778428094,454.36500940635983
stddev,7.4521616583400085,12.707361709685806,5.938535418520848,14.59965835208147,17.066281466837733
min,1.81,25.36,1000,100,420.26
max,9.99,81.56,999.99,99.97,495.76


In [0]:
%sql
select AT as Temperature, PE as Power from pplant

Temperature,Power
14.96,463.26
25.18,444.37
5.11,488.56
20.86,446.48
10.82,473.9
26.27,443.67
15.89,467.35
9.48,478.42
14.64,475.98
11.74,477.5


In [0]:
%sql
select V as ExhaustVacuum, PE as Power from pplant

ExhaustVacuum,Power
41.76,463.26
62.96,444.37
39.4,488.56
57.32,446.48
37.5,473.9
59.44,443.67
43.96,467.35
44.71,478.42
45,475.98
43.56,477.5


In [0]:
%sql
-- TO DO: Replace <FILL_IN> with the appropriate SQL command.
select AP as Pressure, PE as Power from pplant

Pressure,Power
1024.07,463.26
1020.04,444.37
1012.16,488.56
1010.24,446.48
1009.23,473.9
1012.23,443.67
1014.02,467.35
1019.12,478.42
1021.78,475.98
1015.14,477.5


In [0]:
%sql
-- TO DO: Replace <FILL_IN> with the appropriate SQL command.
select RH as Humidity, PE as Power from pplant

Humidity,Power
73.17,463.26
59.08,444.37
92.14,488.56
76.64,446.48
96.62,473.9
58.77,443.67
75.24,467.35
66.43,478.42
41.25,475.98
70.72,477.5


In [0]:
df = sqlContext.table("pplant")
display(df.describe())

summary,AT,V,AP,RH,PE
count,47840,47840,47840,47840,47840
mean,19.651231187290833,54.30580372073524,1013.2590781772577,73.3089778428094,454.36500940635983
stddev,7.4521616583400085,12.707361709685806,5.938535418520848,14.59965835208147,17.066281466837733
min,1.81,25.36,1000,100,420.26
max,9.99,81.56,999.99,99.97,495.76


In [0]:
df_num = df.select(df.AT.cast("float"), df.V.cast("float"), df.AP.cast("float"), df.RH.cast("float"), df.PE.cast("float"))

In [0]:
sqlContext.registerDataFrameAsTable(df_num, "powerdf")

In [0]:
dfplant = sqlContext.table("powerdf")
display(df.describe())

summary,AT,V,AP,RH,PE
count,47840,47840,47840,47840,47840
mean,19.651231187290833,54.30580372073524,1013.2590781772577,73.3089778428094,454.36500940635983
stddev,7.4521616583400085,12.707361709685806,5.938535418520848,14.59965835208147,17.066281466837733
min,1.81,25.36,1000,100,420.26
max,9.99,81.56,999.99,99.97,495.76


In [0]:
# TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = sqlContext.table("powerdf")

vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")

Out[9]: VectorAssembler_77f0a98840c4

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.
# We'll hold out 20% of our data for testing and leave 80% for training
seed = 1800009193
(split20DF, split80DF) = datasetDF.randomSplit([0.2,0.8],seed)

# Let's cache these datasets for performance
testSetDF = split20DF
trainingSetDF = split80DF

In [0]:
# ***** LINEAR REGRESSION MODEL ****

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline

# Let's initialize our linear regression learner
lr = LinearRegression()

# We use explain params to dump the parameters we can use
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [0]:
# Now we set the parameters for the method
lr.setPredictionCol("Predicted_PE")\
  .setLabelCol("PE")\
  .setMaxIter(100)\
  .setRegParam(0.1)


# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()

lrPipeline.setStages([vectorizer, lr])

# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSetDF)

In [0]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept

# The coefficents (i.e., weights) are as follows:
weights = lrModel.stages[1].coefficients

# Create a list of the column names (without PE)
featuresNoLabel = [col for col in datasetDF.columns if col != "PE"]

# Merge the weights and labels
coefficents = sorted(zip(weights, featuresNoLabel))

# Now let's sort the coefficients from greatest absolute weight most to the least absolute weight
sorted(coefficents, key=lambda tup: abs(tup[0]), reverse=True)

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

Linear Regression Equation: y = 435.5639859396408 - (1.9173222368584646 * AT) - (0.2548382144296256 * V) - (0.14707066837222815 * RH) + (0.08003564435144088 * AP)

In [0]:
# Apply our LR model to the test data and predict power output
predictionsAndLabelsDF = lrModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Predicted_PE")

display(predictionsAndLabelsDF)

AT,V,AP,RH,PE,Predicted_PE
1.81,39.42,1026.92,76.97,490.55,492.91808874487697
2.34,39.42,1028.47,69.68,490.34,493.09810270803683
2.71,39.42,1026.66,81.11,489.3,490.5628160224286
3.0,39.64,1011.0,80.14,485.2,488.8400257303371
3.21,38.44,1016.9,86.34,491.35,488.3035685997835
3.21,38.44,1017.11,84.86,492.93,488.53803691954215
3.21,38.44,1017.11,84.86,492.93,488.53803691954215
3.31,39.42,1024.05,84.31,487.19,488.7329053138364
3.38,39.64,1011.0,81.22,488.92,487.95260646977437
3.38,41.31,998.79,97.76,489.11,484.1172401805183


In [0]:
# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
regEval = RegressionEvaluator(predictionCol="Predicted_PE", labelCol="PE", metricName="rmse")

# Run the evaluator on the DataFrame
rmse = regEval.evaluate(predictionsAndLabelsDF)

print("Root Mean Squared Error: %.2f" % rmse)

Root Mean Squared Error: 4.46

In [0]:
# Now let's compute another evaluation metric for our test dataset
r2 = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})

print("r2: {0:.2f}".format(r2))

r2: 0.93

In [0]:
# First we remove the table if it already exists
sqlContext.sql("DROP TABLE IF EXISTS RMSE_Evaluation")
dbutils.fs.rm("dbfs:/user/hive/warehouse/RMSE_Evaluation", True)

# Next we calculate the residual error and divide it by the RMSE
predictionsAndLabelsDF.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmse)).registerTempTable("RMSE_Evaluation")

In [0]:
%sql
SELECT * from RMSE_Evaluation

PE,Predicted_PE,Residual_Error,Within_RSME
490.55,492.91808874487697,-2.368100951908218,-0.5305988859856681
490.34,493.09810270803683,-2.7581063701462085,-0.6179838601265308
489.3,490.5628160224286,-1.262828229459842,-0.28295045918659617
485.2,488.8400257303371,-3.6400135233058677,-0.815584791215318
491.35,488.3035685997835,3.046437503732136,0.68258760016236
492.93,488.53803691954215,4.391955756239099,0.9840656622687459
492.93,488.53803691954215,4.391955756239099,0.9840656622687459
487.19,488.7329053138364,-1.5429028724301475,-0.34570424231105706
488.92,487.95260646977437,0.9674069579600086,0.21675809630275383
489.11,484.1172401805183,4.9927451710441915,1.1186790933180817


In [0]:
%sql
SELECT case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1
            when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3
       end RSME_Multiple, COUNT(*) AS count
FROM RMSE_Evaluation
GROUP BY case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3 end

RSME_Multiple,count
1,6428
3,303
2,2792


In [0]:
pip install mlflow

Python interpreter will be restarted.
Requirement already satisfied: mlflow in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (1.12.0)
Requirement already satisfied: docker>=4.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (4.3.1)
Requirement already satisfied: protobuf>=3.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (3.14.0)
Requirement already satisfied: alembic<=1.4.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (1.4.1)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.8.1)
Requirement already satisfied: click>=7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (7.1.2)
Requirement already satisfied: querystring-parser in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.14.0)
Requirement already satisfied: databricks-cli>=0.8.7 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (0.14.0)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.22.0)
Requirement already satisfied: gunicorn; platform_system != "Windows" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (20.0.4)
Requirement already satisfied: azure-storage-blob in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (12.6.0)
Requirement already satisfied: sqlalchemy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (1.3.20)
Requirement already satisfied: Flask in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (1.1.2)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.0.1)
Requirement already satisfied: pyyaml in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (5.3.1)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.18.1)
Requirement already satisfied: gitpython>=2.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (3.1.11)
Requirement already satisfied: prometheus-flask-exporter in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (0.18.1)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3)
Requirement already satisfied: sqlparse>=0.3.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (0.4.1)
Requirement already satisfied: cloudpickle in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from mlflow) (1.6.0)
Requirement already satisfied: websocket-client>=0.32.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from docker>=4.0.0->mlflow) (0.57.0)
Requirement already satisfied: Mako in /local_disk0/.ephemeral_nfs/envs/pythonEnv-3af835be-64d7-466a-a3a7-27daaaf28bd8/lib/python3.7/site-packages (from alembic<=1.4.1->mlflow) (1.1.3)
Re

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# We can reuse the RegressionEvaluator, regEval, to judge the model based on the best Root Mean Squared Error
# Let's create our CrossValidator with 3 fold cross validation
crossval = CrossValidator(estimator=lrPipeline, evaluator=regEval, numFolds=3)

# Let's tune over our regularization parameter from 0.01 to 0.10
regParam = [x / 100.0 for x in range(1, 11)]

# We'll create a paramter grid using the ParamGridBuilder, and add the grid to the CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, regParam)
             .build())
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
cvModel = crossval.fit(trainingSetDF).bestModel

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
/databricks/spark/python/pyspark/sql/context.py:77: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Now let's use cvModel to compute an evaluation metric for our test dataset: testSetDF
predictionsAndLabelsDF = cvModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Predicted_PE")

# Run the previously created RMSE evaluator, regEval, on the predictionsAndLabelsDF DataFrame
rmseNew = regEval.evaluate(predictionsAndLabelsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2New = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})


print("Original Root Mean Squared Error: {0:2.2f}".format(rmse))
print("New Root Mean Squared Error: {0:2.2f}".format(rmseNew))
print("Old r2: {0:2.2f}".format(r2))
print("New r2: {0:2.2f}".format(r2New))

Original Root Mean Squared Error: 4.46
New Root Mean Squared Error: 4.46
Old r2: 0.93
New r2: 0.93

In [0]:
print("Regularization parameter of the best model: {0:.2f}".format(cvModel.stages[-1]._java_obj.parent().getRegParam()))

Regularization parameter of the best model: 0.01

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.
from pyspark.ml.regression import DecisionTreeRegressor

# Create a DecisionTreeRegressor
dt = DecisionTreeRegressor(maxDepth=2)

dt.setLabelCol("PE")\
  .setPredictionCol("Predicted_PE")\
  .setFeaturesCol("features")\
  .setMaxBins(100)

# Create a Pipeline
dtPipeline = Pipeline()

# Set the stages of the Pipeline
dtPipeline.setStages([vectorizer, dt])

Out[21]: Pipeline_3e4c5a12d4a5

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Let's just reuse our CrossValidator with the new dtPipeline,  RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(dtPipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [2,3])
             .build())

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
dtModel = crossval.fit(trainingSetDF).bestModel

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.

# Now let's use dtModel to compute an evaluation metric for our test dataset: testSetDF
predictionsAndLabelsDF = dtModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Predicted_PE")

# Run the previously created RMSE evaluator, regEval, on the predictionsAndLabelsDF DataFrame
rmseDT = regEval.evaluate(predictionsAndLabelsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2DT = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})

print("LR Root Mean Squared Error: {0:.2f}".format(rmseNew))
print("DT Root Mean Squared Error: {0:.2f}".format(rmseDT))
print("LR r2: {0:.2f}".format(r2New))
print("DT r2: {0:.2f}".format(r2DT))

LR Root Mean Squared Error: 4.46
DT Root Mean Squared Error: 5.07
LR r2: 0.93
DT r2: 0.91

In [0]:
print (dtModel.stages[-1]._java_obj.toDebugString())

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_d67ce2eec31a, depth=3, numNodes=15, numFeatures=4
 If (feature 0 <= 17.795000076293945)
 If (feature 0 <= 11.805000305175781)
 If (feature 0 <= 8.53499984741211)
 Predict: 483.90168508062635
 Else (feature 0 > 8.53499984741211)
 Predict: 476.134397437422
 Else (feature 0 > 11.805000305175781)
 If (feature 0 <= 14.444999694824219)
 Predict: 468.790497807094
 Else (feature 0 > 14.444999694824219)
 Predict: 462.04198863926734
 Else (feature 0 > 17.795000076293945)
 If (feature 0 <= 22.925000190734863)
 If (feature 1 <= 47.33500099182129)
 Predict: 457.6916428573186
 Else (feature 1 > 47.33500099182129)
 Predict: 448.85743321172663
 Else (feature 0 > 22.925000190734863)
 If (feature 1 <= 65.99500274658203)
 Predict: 442.9142964714125
 Else (feature 1 > 65.99500274658203)
 Predict: 434.77679749632665

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.

from pyspark.ml.regression import RandomForestRegressor

# Create a RandomForestRegressor
rf = RandomForestRegressor(numTrees=2, maxDepth=2, seed=42)

rf.setLabelCol("PE")\
  .setPredictionCol("Predicted_PE")\
  .setFeaturesCol("features")\
  .setSeed(100088121)\
  .setMaxDepth(8)\
  .setNumTrees(30)

# Create a Pipeline
rfPipeline = Pipeline()

# Set the stages of the Pipeline
rfPipeline.setStages([vectorizer, rf])

Out[31]: Pipeline_ac82a21e45ed

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.
# Let's just reuse our CrossValidator with the new rfPipeline,  RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(rfPipeline)

# Let's tune over our rf.maxBins parameter on the values 50 and 100, create a paramter grid using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxBins, [50,100])
             .build())

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
rfModel = crossval.fit(trainingSetDF).bestModel

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# TODO: Replace <FILL_IN> with the appropriate code.

# Now let's use rfModel to compute an evaluation metric for our test dataset: testSetDF
predictionsAndLabelsDF = rfModel.transform(testSetDF).select("AT", "V", "AP", "RH", "PE", "Predicted_PE")

# Run the previously created RMSE evaluator, regEval, on the predictionsAndLabelsDF DataFrame
rmseRF = regEval.evaluate(predictionsAndLabelsDF)

# Now let's compute the r2 evaluation metric for our test dataset
r2RF = regEval.evaluate(predictionsAndLabelsDF, {regEval.metricName: "r2"})

print("LR Root Mean Squared Error: {0:.2f}".format(rmseNew))
print("DT Root Mean Squared Error: {0:.2f}".format(rmseDT))
print("RF Root Mean Squared Error: {0:.2f}".format(rmseRF))
print("LR r2: {0:.2f}".format(r2New))
print("DT r2: {0:.2f}".format(r2DT))
print("RF r2: {0:.2f}".format(r2RF))

LR Root Mean Squared Error: 4.46
DT Root Mean Squared Error: 5.07
RF Root Mean Squared Error: 3.35
LR r2: 0.93
DT r2: 0.91
RF r2: 0.96

In [0]:
print (rfModel.stages[-1]._java_obj.toDebugString())

RandomForestRegressionModel: uid=RandomForestRegressor_400845a79488, numTrees=30, numFeatures=4
 Tree 0 (weight 1.0):
 If (feature 2 <= 1014.8150024414062)
 If (feature 1 <= 46.19499969482422)
 If (feature 3 <= 79.25500106811523)
 If (feature 0 <= 15.305000305175781)
 If (feature 0 <= 9.234999656677246)
 If (feature 1 <= 37.71500015258789)
 If (feature 0 <= 8.505000114440918)
 If (feature 0 <= 7.644999980926514)
 Predict: 481.55999755859375
 Else (feature 0 > 7.644999980926514)
 Predict: 480.6300048828125
 Else (feature 0 > 8.505000114440918)
 If (feature 2 <= 1013.7750244140625)
 Predict: 479.08334011501734
 Else (feature 2 > 1013.7750244140625)
 Predict: 478.07000732421875
 Else (feature 1 > 37.71500015258789)
 If (feature 0 <= 5.734999895095825)
 If (feature 3 <= 77.4900016784668)
 Predict: 493.1104855421113
 Else (feature 3 > 77.4900016784668)
 Predict: 486.8299865722656
 Else (feature 0 > 5.734999895095825)
 If (feature 0 <= 8.505000114440918)
 Predict: 486.47494147805605
 Else (feature 0 > 8.505000114440918)
 Predict: 482.2365394005409
 Else (feature 0 > 9.234999656677246)
 If (feature 1 <= 44.89500045776367)
 If (feature 1 <= 41.18499946594238)
 If (feature 0 <= 13.795000076293945)
 Predict: 474.58823623443516
 Else (feature 0 > 13.795000076293945)
 Predict: 467.6337874373035
 Else (feature 1 > 41.18499946594238)
 If (feature 0 <= 11.494999885559082)
 Predict: 477.05419011269845
 Else (feature 0 > 11.494999885559082)
 Predict: 467.3486516716775
 Else (feature 1 > 44.89500045776367)
 If (feature 3 <= 75.44499969482422)
 If (feature 2 <= 1014.260009765625)
 Predict: 463.9449966430664
 Else (feature 2 > 1014.260009765625)
 Predict: 460.6655578613281
 Else (feature 3 > 75.44499969482422)
 If (feature 0 <= 13.394999980926514)
 Predict: 466.39109971788196
 Else (feature 0 > 13.394999980926514)
 Predict: 464.5899963378906
 Else (feature 0 > 15.305000305175781)
 If (feature 1 <= 43.135000228881836)
 If (feature 2 <= 1008.7850036621094)
 If (feature 0 <= 21.020000457763672)
 If (feature 0 <= 19.045000076293945)
 Predict: 461.78507450321416
 Else (feature 0 > 19.045000076293945)
 Predict: 457.3744918159817
 Else (feature 0 > 21.020000457763672)
 If (feature 2 <= 1007.9049987792969)
 Predict: 451.6854571718158
 Else (feature 2 > 1007.9049987792969)
 Predict: 440.69000244140625
 Else (feature 2 > 1008.7850036621094)
 If (feature 0 <= 17.869999885559082)
 If (feature 1 <= 35.76499938964844)
 Predict: 461.7774937947591
 Else (feature 1 > 35.76499938964844)
 Predict: 465.6120421117666
 Else (feature 0 > 17.869999885559082)
 If (feature 2 <= 1011.3250122070312)
 Predict: 464.202954378995
 Else (feature 2 > 1011.3250122070312)
 Predict: 460.69601123580514
 Else (feature 1 > 43.135000228881836)
 If (feature 0 <= 23.505000114440918)
 If (feature 2 <= 1010.9749755859375)
 If (feature 0 <= 21.304999351501465)
 Predict: 454.2085882288809
 Else (feature 0 > 21.304999351501465)
 Predict: 447.7133353904442
 Else (feature 2 > 1010.9749755859375)
 If (feature 0 <= 19.40499973297119)
 Predict: 460.1234398158729
 Else (feature 0 > 19.40499973297119)
 Predict: 455.2432906820167
 Else (feature 0 > 23.505000114440918)
 If (feature 2 <= 1005.1549987792969)
 Predict: 429.84749603271484
 Else (feature 2 > 1005.1549987792969)
 If (feature 2 <= 1010.9749755859375)
 Predict: 444.9029778094331
 Else (feature 2 > 1010.9749755859375)
 Predict: 440.2508140096859
 Else (feature 3 > 79.25500106811523)
 If (feature 0 <= 9.864999771118164)
 If (feature 0 <= 8.079999923706055)
 If (feature 1 <= 41.60000038146973)
 If (feature 3 <= 92.63500213623047)
 If (feature 1 <= 39.875)
 Predict: 483.7156133846361
 Else (feature 1 > 39.875)
 Predict: 487.1441436118268
 Else (feature 3 > 92.63500213623047)
 If (feature 0 <= 6.7799999713897705)
 Predict: 488.70565668741864
 Else (feature 0 > 6.7799999713897705)
 Predict: 483.24739929199217
 Else (feature 1 > 41.60000038146973)
 If (feature 3 <= 86.54500198364258)
 If (feature 0 <= 5.734999895095825)
 Predict: 486.19556003146704
 

Wow! So our best model is in fact our Random Forest tree model which uses an ensemble of 30 Trees with a depth of 8 to construct a better model than the single decision tree.

# Further reading


1.Steps for a Developer to Learn Apache Spark™ with Delta Lake [link](https://databricks.com/p/ebook/learn-apache-spark-with-delta-lake utm_medium=cpc&utm_source=bing&utm_campaign=392642117&utm_offer=learn-apache-spark-with-delta lake&utm_content=ebook&utm_term=%2Bspark&msclkid=e89c97e97c3b11a24c5dc85a6642b6bd).

2.MLlib: Machine Learning in Apache Spark [link](https://jmlr.org/papers/volume17/15-237/15-237.pdf).
 
3.Hands-On Deep Learning with Apache Spark [link](https://lib-ebooks.com/hands-on-deep-learning-with-apache-spark/)
 
 
 
# Summary
In this tutorial, you discovered how to Extract, Transform and Load(ETL) data. focused on a Spark MAchine learning API. 

Specifically, you learned:

* How to do ETL.
* How to create a datadrame and explore it.
* How to prepare a data and develop a model.
* How to choose the best model.


## Next Step

There is still much room to improve like try to do same job using sparkdl pretrain models.